**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [55]:
 pip install scikit-video

In [0]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML

import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense, Flatten, Dropout
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

epsilon is essential because it's the parameter of the $\epsilon$-greedy policy, $\epsilon$ controls the trade off between exploration and exploitation, $\epsilon = 0$ means the agent will choose the best action based on history, and the actions will be dependant to the cumulated knowledge and we will have a big bias, $\epsilon = 1$ means the agent will take random action all the time, exploration is important because during this part the agent learns new things and strenghens its strategy. 

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=11 # set small when debugging
epochs_test=11 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.


position : 
- An array containing the cells where the rat can go position[x,y]=0, the boundries where the rat can't go position[x,y]=-1 and the actual position of the rat position[x,y]=1.

Board: 
- An array containing the reward of each cell of the grid, the rat gets a reward board[x,y] when it's in the cell [x,y]


## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        a = np.random.randint(0, self.n_action, size=1)[0]
        return a

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        ##### FILL IN HERE
        s= env.reset()
        game_over = False # This assumes that the games won't end
        win = 0
        lose = 0
         
        while not game_over:
            #I choose train = False, because in test the action must be dependent to the previous cumulated knowledge
            a = agent.act(s, train= False) 
            s,r,game_over =env.act(a)
            if r>0 :
                win +=r
            if r < 0:
                lose+=-r
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [63]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 10.5/15.0. Average score (-4.5)
Win/lose count 14.0/8.0. Average score (0.75)
Win/lose count 8.0/9.0. Average score (0.16666666666666666)
Win/lose count 10.0/15.0. Average score (-1.125)
Win/lose count 11.0/17.0. Average score (-2.1)
Win/lose count 8.5/11.0. Average score (-2.1666666666666665)
Win/lose count 10.0/15.0. Average score (-2.5714285714285716)
Win/lose count 10.0/20.0. Average score (-3.5)
Win/lose count 11.0/17.0. Average score (-3.7777777777777777)
Win/lose count 7.0/10.0. Average score (-3.7)
Win/lose count 9.5/16.0. Average score (-3.9545454545454546)
Final score: -3.9545454545454546


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




We have in the case $T=\infty$:
\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}
\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[r(s,a) + \sum_{t=1}^{\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}
\begin{equation*}Q^\pi(s,a)=r(s,a) + \gamma E_{p^{\pi}}[ \sum_{t=1}^{\infty}\gamma^{t-1}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}
\begin{equation*}Q^\pi(s,a)=r(s,a) + \gamma E_{p^{\pi}}[E_{p^{\pi}}[ \sum_{t=0}^{\infty}\gamma^{t}r(s_{t+1},a_{t+1})|s_{1}=s',a_{1}=a']|s_{0}=s,a_{0}=a] \>
\end{equation*}

\begin{equation*}Q^\pi(s,a)=  E_{p^{\pi}}[r(s,a) +\gamma E_{p^{\pi}}[ \sum_{t=0}^{\infty}\gamma^{t}r(s_{t+1},a_{t+1})|s_{1}=s',a_{1}=a']|s_{0}=s,a_{0}=a] \> 
\end{equation*}

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}
Let $\pi^*$ be the optimal policy,
\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

\begin{equation*}
Q^{\pi}(s,a)=r(s,a) + \gamma \sum_{s'\in S, a'\in A} P(s'|s,a)\pi(a'|s') Q^{\pi}(s',a')
\end{equation*}

\begin{equation*}
Q^{\pi^*}(s,a)=r(s,a) + \gamma \sum_{s'\in S, a'\in A} P(s'|s,a)\pi^*(a'|s') Q^{\pi^*}(s',a')
\end{equation*}
We know that the optimal policy is deterministic and:
$$\pi^*(a'|s') = \left\{\begin{aligned} &1 &&\textrm{if  $\quad a'= argmax(Q^*(s',a'))$}\\&0 &&\textrm{otherwise }\end{aligned}\right.$$
So Q becomes:
\begin{equation*}
Q^{\pi^*}(s,a)=r(s,a) + \gamma \sum_{s'\in S, a'\in A} P(s'|s,a)\max_{a'} Q^{\pi^*}(s',a')
\end{equation*}

\begin{equation*}
Q^{*}(s,a)= Q^{\pi ^*}(s,a)= E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
- Objectif function:
The optimal Q-function must verify the optimal Bellman equation (Eq.2), So if the neural network perfectly converge, the equation must be verified, so we can choose for the loss the average square error between the values predicted by the network and the target output for each actions.


***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
import numpy as np
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        if len(self.memory)< self.max_memory:
            self.memory.append(m)
        else : 
            self.memory[np.random.randint(len(self.memory))]=m
        

    def random_access(self):
        
        return(self.memory[np.random.randint(len(self.memory))])

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s.reshape(1,s.shape[0],s.shape[1],s.shape[2])),1)
        pass

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            ######## FILL IN
            s, n_s, a, r, game_over = self.memory.random_access()
            #print(a)
            input_states[i] = s
            target_q[i,:] = self.model.predict(s.reshape((1,5,5,self.n_state)))
            if game_over:
                ######## FILL IN
                target_q[i, int(a)] = r
                
            else:
                ######## FILL IN
                target_q[i, int(a)] = r +self.discount*np.max(self.model.predict(n_s.reshape((1,5,5,self.n_state))))
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -8, 8)

        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        model = Sequential()
        model.add(Flatten(input_shape=(5, 5, self.n_state)))
        model.add(Dense(32 , activation='relu'))
        model.add(Dense(4))
        ####### FILL IN
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [67]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epoch=80, prefix='fc_train')
HTML(display_videos('fc_train70.mp4'))

Epoch 000/080 | Loss 0.0028 | Win/lose count 3.0/4.0 (-1.0)
Epoch 001/080 | Loss 0.0025 | Win/lose count 0.5/4.0 (-3.5)
Epoch 002/080 | Loss 0.0029 | Win/lose count 4.0/4.0 (0.0)
Epoch 003/080 | Loss 0.0083 | Win/lose count 3.5/2.0 (1.5)
Epoch 004/080 | Loss 0.0129 | Win/lose count 4.5/2.0 (2.5)
Epoch 005/080 | Loss 0.0277 | Win/lose count 4.0/0 (4.0)
Epoch 006/080 | Loss 0.0163 | Win/lose count 2.5/3.0 (-0.5)
Epoch 007/080 | Loss 0.0112 | Win/lose count 7.0/5.0 (2.0)
Epoch 008/080 | Loss 0.0568 | Win/lose count 4.0/4.0 (0.0)
Epoch 009/080 | Loss 0.0418 | Win/lose count 1.5/5.0 (-3.5)
Epoch 010/080 | Loss 0.0258 | Win/lose count 2.0/3.0 (-1.0)
Epoch 011/080 | Loss 0.0521 | Win/lose count 4.0/4.0 (0.0)
Epoch 012/080 | Loss 0.0332 | Win/lose count 2.5/2.0 (0.5)
Epoch 013/080 | Loss 0.0276 | Win/lose count 5.0/5.0 (0.0)
Epoch 014/080 | Loss 0.0159 | Win/lose count 4.5/4.0 (0.5)
Epoch 015/080 | Loss 0.1873 | Win/lose count 5.0/5.0 (0.0)
Epoch 016/080 | Loss 0.0128 | Win/lose count 2.5/2.0 

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
        model = Sequential()
        model.add(Conv2D(32, kernel_size=(2, 2),
                         activation='relu',
                         input_shape=(5,5,self.n_state)))
        model.add(Conv2D(64, (2, 2), activation='relu'))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(128, activation='relu'))
        
        model.add(Dense(4))
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [69]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epoch=80,prefix='cnn_train')
HTML(display_videos('cnn_train70.mp4'))

Epoch 000/080 | Loss 0.1126 | Win/lose count 4.0/5.0 (-1.0)
Epoch 001/080 | Loss 0.0351 | Win/lose count 4.0/10.0 (-6.0)
Epoch 002/080 | Loss 0.0227 | Win/lose count 4.5/2.0 (2.5)
Epoch 003/080 | Loss 0.0249 | Win/lose count 3.5/1.0 (2.5)
Epoch 004/080 | Loss 0.0107 | Win/lose count 1.0/2.0 (-1.0)
Epoch 005/080 | Loss 0.0042 | Win/lose count 1.5/1.0 (0.5)
Epoch 006/080 | Loss 0.0288 | Win/lose count 4.0/5.0 (-1.0)
Epoch 007/080 | Loss 0.0026 | Win/lose count 4.0/3.0 (1.0)
Epoch 008/080 | Loss 0.0021 | Win/lose count 5.0/3.0 (2.0)
Epoch 009/080 | Loss 0.0050 | Win/lose count 2.5/2.0 (0.5)
Epoch 010/080 | Loss 0.0035 | Win/lose count 2.5/5.0 (-2.5)
Epoch 011/080 | Loss 0.0060 | Win/lose count 2.5/0 (2.5)
Epoch 012/080 | Loss 0.0031 | Win/lose count 3.5/1.0 (2.5)
Epoch 013/080 | Loss 0.0039 | Win/lose count 4.5/3.0 (1.5)
Epoch 014/080 | Loss 0.0186 | Win/lose count 6.0/1.0 (5.0)
Epoch 015/080 | Loss 0.0021 | Win/lose count 5.5/3.0 (2.5)
Epoch 016/080 | Loss 0.0112 | Win/lose count 8.0/0 (

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [70]:
env = Environment(grid_size=size, max_time=T,temperature=0.8)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.3, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.3, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs=21,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs=21,prefix='fc_test')

Test of the CNN
Win/lose count 24.5/1.0. Average score (23.5)
Win/lose count 15.5/0. Average score (19.5)
Win/lose count 23.0/0. Average score (20.666666666666668)
Win/lose count 18.5/0. Average score (20.125)
Win/lose count 10.5/0. Average score (18.2)
Win/lose count 24.5/0. Average score (19.25)
Win/lose count 30.0/1.0. Average score (20.642857142857142)
Win/lose count 28.5/2.0. Average score (21.375)
Win/lose count 15.5/1.0. Average score (20.61111111111111)
Win/lose count 26.5/0. Average score (21.2)
Win/lose count 9.5/0. Average score (20.136363636363637)
Win/lose count 49.0/1.0. Average score (22.458333333333332)
Win/lose count 38.0/0. Average score (23.653846153846153)
Win/lose count 28.0/2.0. Average score (23.821428571428573)
Win/lose count 5.5/0. Average score (22.6)
Win/lose count 8.0/0. Average score (21.6875)
Win/lose count 9.0/0. Average score (20.941176470588236)
Win/lose count 4.0/0. Average score (20.0)
Win/lose count 2.0/0. Average score (19.05263157894737)
Win/lose c

In [71]:
HTML(display_videos('cnn_test20.mp4'))

In [72]:
HTML(display_videos('fc_test20.mp4'))

- The model with CNN layers is more efficient than the model with FC layers, for all the values of the temperature. This is happening because the CNN allows a better spatial representation hence a better decision.
- But the videos show that the two models are suffering from the issue of the agent oscillating over a position, setting the epsilon ratio high helps avoid this issue and introduces more exploration.
- Setting the Temperature high (0.8) allows more exploration, because the agent can follow a path made only by red cases.


***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0
    rate=0.99
    k = 1/8
    epsilon_final = 0.1
    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False
        new_eps = agent.epsilon/(1+k*e)
        if new_eps > epsilon_final:
            agent.set_epsilon(new_eps)
        else :
            agent.set_epsilon(epsilon_final) 

        win = 0
        lose = 0
        
        while not game_over:
            # The agent performs an action
            action = agent.act(state)
            
            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = round(win + reward,2)
            if reward < 0:
                lose = round(lose -reward,2)

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += round(win-lose,2)

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))
        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = 0
        if train:
            reward = - self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.5
        reward += self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus
        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state
    


In [74]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=100, temperature=0.4)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.7, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent_cnn, env, epoch=90, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore80.mp4'))

Epoch 000/090 | Loss 0.0376 | Win/lose count 9.0/36.5 (-27.5)
Epoch 001/090 | Loss 0.0890 | Win/lose count 8.0/42.0 (-34.0)
Epoch 002/090 | Loss 0.0803 | Win/lose count 6.5/38.5 (-32.0)
Epoch 003/090 | Loss 0.0502 | Win/lose count 8.5/35.5 (-27.0)
Epoch 004/090 | Loss 0.0603 | Win/lose count 22.5/16.0 (6.5)
Epoch 005/090 | Loss 0.0224 | Win/lose count 20.5/17.5 (3.0)
Epoch 006/090 | Loss 0.0421 | Win/lose count 22.5/14.0 (8.5)
Epoch 007/090 | Loss 0.0391 | Win/lose count 17.5/18.0 (-0.5)
Epoch 008/090 | Loss 0.0699 | Win/lose count 19.0/19.5 (-0.5)
Epoch 009/090 | Loss 0.1003 | Win/lose count 16.5/23.5 (-7.0)
Epoch 010/090 | Loss 0.0963 | Win/lose count 29.0/11.5 (17.5)
Epoch 011/090 | Loss 0.0714 | Win/lose count 21.0/20.0 (1.0)
Epoch 012/090 | Loss 0.0816 | Win/lose count 21.5/13.0 (8.5)
Epoch 013/090 | Loss 0.0925 | Win/lose count 25.5/11.5 (14.0)
Epoch 014/090 | Loss 0.0862 | Win/lose count 21.5/15.0 (6.5)
Epoch 015/090 | Loss 0.3092 | Win/lose count 24.5/14.0 (10.5)
Epoch 016/090 

In [75]:
# Training DQN_FC
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent_fc = DQN_FC(size, lr=.1, epsilon = 0.7, memory_size=2000, batch_size = 32)
train_explore(agent_fc, env, epoch=90, prefix='fc_train_explore')
HTML(display_videos('fc_train_explore80.mp4'))


Epoch 000/090 | Loss 0.0472 | Win/lose count 7.0/16.0 (-9.0)
Epoch 001/090 | Loss 0.0775 | Win/lose count 6.0/10.0 (-4.0)
Epoch 002/090 | Loss 0.0457 | Win/lose count 6.5/8.0 (-1.5)
Epoch 003/090 | Loss 0.0123 | Win/lose count 4.5/7.0 (-2.5)
Epoch 004/090 | Loss 0.0132 | Win/lose count 6.5/8.0 (-1.5)
Epoch 005/090 | Loss 0.0054 | Win/lose count 7.5/4.0 (3.5)
Epoch 006/090 | Loss 0.0073 | Win/lose count 5.5/7.0 (-1.5)
Epoch 007/090 | Loss 0.0119 | Win/lose count 5.0/2.0 (3.0)
Epoch 008/090 | Loss 0.0249 | Win/lose count 3.5/1.0 (2.5)
Epoch 009/090 | Loss 0.0166 | Win/lose count 3.0/2.0 (1.0)
Epoch 010/090 | Loss 0.0130 | Win/lose count 8.0/2.0 (6.0)
Epoch 011/090 | Loss 0.0059 | Win/lose count 11.5/4.0 (7.5)
Epoch 012/090 | Loss 0.3660 | Win/lose count 9.0/3.0 (6.0)
Epoch 013/090 | Loss 0.0100 | Win/lose count 2.0/2.0 (0.0)
Epoch 014/090 | Loss 0.0167 | Win/lose count 10.0/4.0 (6.0)
Epoch 015/090 | Loss 0.0120 | Win/lose count 2.0/4.0 (-2.0)
Epoch 016/090 | Loss 0.3607 | Win/lose count 

In [77]:
# Evaluation DQN_FC
test(agent_fc,env,epochs_test,prefix='fc_test_explore')
HTML(display_videos('fc_test_explore10.mp4'))

Win/lose count 1.0/0. Average score (1.0)
Win/lose count 0/0. Average score (0.5)
Win/lose count 3.5/1.0. Average score (1.1666666666666667)
Win/lose count 7.5/0. Average score (2.75)
Win/lose count 7.0/0. Average score (3.6)
Win/lose count 2.0/0. Average score (3.3333333333333335)
Win/lose count 8.0/0. Average score (4.0)
Win/lose count 6.0/0. Average score (4.25)
Win/lose count 5.5/0. Average score (4.388888888888889)
Win/lose count 0.5/0. Average score (4.0)
Win/lose count 2.0/0. Average score (3.8181818181818183)
Final score: 3.8181818181818183


***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***